In [1]:
#conda list


In [2]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])

DL_test


In [3]:
import torch
import numpy as np

In [4]:
x = torch.tensor([1.,2,3])
w = torch.tensor([4.,5,6],requires_grad = True)
b = torch.tensor(5.,requires_grad = True)
torch.dot(w,x) + b

tensor(37., grad_fn=<AddBackward0>)

In [5]:
y = torch.dot(w,x) + b
y


tensor(37., grad_fn=<AddBackward0>)

In [6]:
y.backward()

In [7]:
print(w.grad)

tensor([1., 2., 3.])


### Broadcasting_test


In [8]:
x = torch.randn(5,2)
y = torch.rand(2)
print(x)
print(y)
x + y



tensor([[ 1.9948,  0.3287],
        [-0.2385, -0.7055],
        [ 1.3495,  0.4530],
        [-0.4426,  0.6137],
        [-0.5841,  1.0668]])
tensor([0.8646, 0.9037])


tensor([[2.8595, 1.2324],
        [0.6261, 0.1981],
        [2.2141, 1.3567],
        [0.4221, 1.5173],
        [0.2805, 1.9705]])

# Linear Regression


In [9]:

inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]],dtype = 'float32')

# 5 (samples) * 3 (features)

In [10]:
targets = np.array([[56,70],[81,101],[119,133],[22,37],[103,119]],dtype  = 'float32')

# 5 (samples) * 2 (outputs)

In [11]:
### convert to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [12]:
w = torch.randn(2,3,requires_grad = True)
b = torch.randn(2,requires_grad = True)

print(w)
print(b)

tensor([[ 2.9732e-01, -5.1109e-01,  5.8895e-01],
        [-1.2988e-03, -5.1069e-01,  2.0479e+00]], requires_grad=True)
tensor([ 2.5938, -0.1764], requires_grad=True)


In [13]:
def model(x):
    return x @ w.t() + b

def mse(a,b):
    diff = a - b
    return torch.sum(diff*diff)/ diff.numel()

In [14]:
preds = model(inputs)
loss = mse(targets,preds)
loss

tensor(3676.5962, grad_fn=<DivBackward0>)

In [15]:
for i in range(100):
    preds = model(inputs)
    loss = mse(targets,preds)
    
    if loss > 0:
        loss.backward()
        with torch.no_grad():
            w -= w.grad * 1e-4
            b -= b.grad * 1e-4
            w.grad.zero_()
            b.grad.zero_()

In [16]:
preds = model(inputs)
loss = mse(targets,preds)
loss

tensor(15.7450, grad_fn=<DivBackward0>)

In [17]:
preds

tensor([[ 57.5332,  69.8332],
        [ 82.6328, 104.5287],
        [117.1409, 124.9474],
        [ 20.7220,  34.8498],
        [103.3816, 127.0778]], grad_fn=<AddBackward0>)

 # Linear Regression using pytorch built ins
 

In [18]:
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [19]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [20]:
train_ds = TensorDataset(inputs, targets)
train_dl = DataLoader(train_ds, 2, shuffle=True)

In [21]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3135, -0.1449, -0.4391],
        [ 0.4151,  0.5764,  0.3371]], requires_grad=True)
Parameter containing:
tensor([ 0.4064, -0.0021], requires_grad=True)


In [22]:
loss_fn = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [23]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            preds = model(xb)
            loss = loss_fn(preds,yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [24]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 343.7706
Epoch [20/100], Loss: 3.9406
Epoch [30/100], Loss: 3.9835
Epoch [40/100], Loss: 36.1574
Epoch [50/100], Loss: 3.4299
Epoch [60/100], Loss: 6.4957
Epoch [70/100], Loss: 2.6752
Epoch [80/100], Loss: 1.7584
Epoch [90/100], Loss: 3.7881
Epoch [100/100], Loss: 30.7923
